In [11]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra, StatsFuns, FixedEffectModels
using LaTeXTabulars, NBInclude
using BlockDiagonals

In [96]:
#== Data Parameters ==#
Random.seed!(6789998212);
T = 100; # Markets
J = 6;   # Products
L = 80;  # Simulated individuals

# Inner Loop Parameters
tol_in = 5e-5;   # Initial (global) contraction tolerance
tol_nm = 8e-15;  # Newton Method (local) tolerance
max_iter = 2000; # Inner loop limit

In [20]:
# DataFrame
df = DataFrame(CSV.File("../data/ps1_ex4.csv"));
df = df[df[!, :market] .<= T, :];

# Inputs
s_jt = Array(df[!,[:shares]]); # observed market shares
x_jt = Array(df[!,[:p, :x]]);  # product atributes
z_jt = Array(df[!,[:z1, :z2, :z3, :z4, :z5, :z6, :x]]); # included and excluded instruments
v = randn(2,L); # simulated individuals

In [95]:
# Inner Loop Functions
function predict_shares(Γ, δ_jt, x_jt, v)
    #log-sum-exp trick to avoid overflow: normalize V = u_ijt by max_j u_ijt
    V = reshape((δ_jt .+ x_jt * Γ * v),J,T,L)
    norm = max.(zeros(1,T,L), maximum(V, dims = 1))
    
    numerator = exp.(V .- norm)
    denominator = exp.(-norm) .+ sum(numerator, dims = 1)
    π_jtl = numerator ./ denominator
    
    return π_jtl
end

function inv_jacobian_ξ(π_jtl) # returns inverse Jacobian of ξ(δ,θ) = s - π(δ,θ) --> ∂ξ_∂δ = -∂π_∂δ
    ∂π_∂δ_store = Matrix{Float64}[]
    for t = 1:T    
        ∂π_∂δ = (diagm(vec(sum(π_jtl[:,t,:], dims = 2))) - π_jtl[:,t,:] * π_jtl[:,t,:]') ./ vec(sum(π_jtl, dims = 3)[:,t,:])
        push!(∂π_∂δ_store, inv(∂π_∂δ))
        end
    ∂π_∂δ = BlockDiagonal(∂π_∂δ_store)
    
    return -∂π_∂δ
end

function solve_delta(Γ, s_jt, x_jt, v)
    δ₀ = ones(J * T)
    π_jtl = zeros(J * T, L)
    err = 1000
    iter = 0
    iter_in = 0
    iter_nm = 0
    while (err > tol_nm) && (iter < max_iter)
        iter += 1
        π_jtl = predict_shares(Γ,δ₀,x_jt,v)
        if (err > tol_in) # global contraction
            δ_jt = δ₀ + log.(s_jt) - log.(vec(mean(π_jtl, dims = 3)))
            iter_in += 1
        else # local Newton Method
            δ_jt = δ₀ - 0.65 * inv_jacobian_ξ(π_jtl) * (log.(s_jt) - log.(vec(mean(π_jtl, dims = 3))))
            iter_nm += 1
        end
        err = maximum(abs.(δ_jt - δ₀))
        δ₀ = δ_jt
    end
    
    return δ₀, π_jtl
end

function resid(δ_jt, x_jt, z_jt)
    θ₁ = inv(x_jt' * z_jt * w * z_jt' * x_jt) * (x_jt' * z_jt * w * z_jt' * δ_jt)
    ξ_jt = δ_jt - (x_jt * θ₁)
    
    return ξ_jt, θ₁
end

resid (generic function with 1 method)

In [15]:
# Outer Loop Functions
function gmm_objective(ξ_jt, z_jt)
    g = (z_jt' * ξ_jt)
    G = g' * w * g
    
    return G[1]
end

function outer_loop(θ₂, s_jt, x_jt, z_jt, v)
    Γ = [θ₂[1] 0; θ₂[2] θ₂[3]]
    δ_jt, π_jtl = solve_delta(Γ, s_jt, x_jt, v)
    ξ_jt, θ₁ = resid(δ_jt, x_jt, z_jt)
    G = gmm_objective(ξ_jt, z_jt)
    
    return G, ξ_jt, θ₁, π_jtl
end

function f(θ₂)
    f, ξ_jt, θ₁, π_jtl = outer_loop(θ₂, s_jt, x_jt, z_jt, v)
    
    return f
end

function gmm_gradient(θ₂, ξ_jt, π_jtl, x_jt, z_jt, v)
    # Implicit function theorem: ∂G_∂θ = (∂G_∂ξ * ∂ξ_∂θ)' = (∂G_∂ξ * (-1 * ∂π_∂ξ⁻¹ * ∂π_∂θ))'
    
    ∂G_∂ξ = 2 * (z_jt' * ξ_jt)' * w * z_jt'

    # Jacobian    
    x_t = reshape(x_jt, J, T, 2)
    ∂ξ_∂θ_store = []
    for t = 1:T    
        ∂π_∂γ₁₁ = π_jtl[:,t,:] .* (x_t[:,t,1] .- (π_jtl[:,t,:]' * x_t[:,t,1])') .* v[1,:]'
        ∂π_∂γ₂₁ = π_jtl[:,t,:] .* (x_t[:,t,2] .- (π_jtl[:,t,:]' * x_t[:,t,2])') .* v[1,:]'
        ∂π_∂γ₂₂ = π_jtl[:,t,:] .* (x_t[:,t,2] .- (π_jtl[:,t,:]' * x_t[:,t,2])') .* v[2,:]'
        ∂π_∂θ = hcat(mean(∂π_∂γ₁₁,dims = 2), mean(∂π_∂γ₂₁,dims = 2), mean(∂π_∂γ₂₂,dims = 2))
    
        ∂π_∂ξ⁻¹ = inv((diagm(vec(sum(π_jtl[:,t,:], dims = 2))) - π_jtl[:,t,:] * π_jtl[:,t,:]') / L)
        ∂ξ_∂θ_t = -1 * ∂π_∂ξ⁻¹ * ∂π_∂θ
        
        push!(∂ξ_∂θ_store, ∂ξ_∂θ_t)
    end
    ∂ξ_∂θ = reduce(vcat,∂ξ_∂θ_store)
    
    ∂G_∂θ = (∂G_∂ξ * ∂ξ_∂θ)'
    
    return ∂G_∂θ
end

function ∇(g, θ₂)
    f, ξ_jt, θ₁, π_jtl = outer_loop(θ₂, s_jt, x_jt, z_jt, v)
    
    g .= vec(gmm_gradient(θ₂, ξ_jt, π_jtl, x_jt, z_jt, v))
end

∇ (generic function with 1 method)

In [97]:
# GMM Step 1:
params0 = ones(3)
w = inv(z_jt' * z_jt)

@time o_1 = optimize(f, ∇, params0, LBFGS(), Optim.Options(show_trace=true, show_every=10))
G, ξ_jt, θ₁, π_jtl = outer_loop(o_1.minimizer, s_jt, x_jt, z_jt, v)
println("θ₁: ", θ₁, "\nθ₂: ", o_1.minimizer)

# GMM Step 2:
w = inv((z_jt .* ξ_jt )' * (z_jt .* ξ_jt))

@time o_2 = optimize(f, ∇, o_1.minimizer, LBFGS(), Optim.Options(show_trace=true, show_every=10))
G, ξ_jt, θ₁, π_jtl = outer_loop(o_1.minimizer, s_jt, x_jt, z_jt, v)
println("θ₁: ", θ₁, "\nθ₂: ", o_2.minimizer)

Iter     Function value   Gradient norm 
     0     6.954119e+02     1.013296e+02
 * time: 0.0
 30.455109 seconds (17.89 M allocations: 47.045 GiB, 3.39% gc time, 0.98% compilation time)
θ₁: [-0.6045380558572608; -0.18869255804226323;;]
θ₂: [1.728336027144186, -0.12342452165307399, -0.019986134921770166]
Iter     Function value   Gradient norm 
     0     1.290784e+02     3.564592e-01
 * time: 0.0
 18.270445 seconds (11.99 M allocations: 28.223 GiB, 3.41% gc time)
θ₁: [-0.571879565099018; -0.19331497422578403;;]
θ₂: [1.5903233682091689, -0.13118771446254024, -0.01840352679379876]


In [81]:
# Elasticities
θ₂ = o_2.minimizer
Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]]

δ_jt, π_jtl = solve_delta(Γ, s_jt, x_jt, v)
x_t = reshape(x_jt, J, T, 2)
s_t = reshape(s_jt, J, T)

αᵢ = θ₁[1] .+ θ₂[1] .* v[1,:]

η = [] 
for t in 1:T
    η_t = zeros(6,6)    
    for j in 1:6 
        for k in 1:6
            if j == k
                η_t[j,k] = - mean(αᵢ .* π_jtl[j,t,:] .* (1 .- π_jtl[j,t,:])) * x_t[j,t,1] / s_t[j,t]
            else
                η_t[j,k] = mean(αᵢ .* π_jtl[j,t,:] .* π_jtl[k,t,:]) * x_t[k,t,1] / s_t[j,t] 
            end
        end
    end
    push!(η,η_t)
end

display(mean(η))

6×6 Matrix{Float64}:
  0.00481492   -0.000861015  -0.0152794  -0.0131148  -0.0238666  -0.0266219
 -0.00129486    0.00479095   -0.0149898  -0.0131016  -0.0232241  -0.0280811
 -0.000336514  -0.000146311  -0.156318    0.0377787   0.260481    0.268805
 -0.000412698  -0.000167494   0.0515264  -0.482733    0.326881    0.420136
 -0.000165373  -0.00017912    0.167326    0.0598248  -0.359199    0.477404
 -0.000198954  -8.27704e-5    0.0967665   0.0885238   0.364493   -0.285644

In [49]:
params0 = ones(3)
w = inv(z_jt' * z_jt)
Γ = [2 -2.3; 0 -0.3]
#Γ = [2 -0.1; 0 -0.2]

δ_jt, π_jtl = solve_delta(Γ, s_jt, x_jt, v);